## 02/03/2019

- Boxplots of feature values for fractionated/non-fractionated segments - trying to identify which ones are able to distinguish between the classes.

In [1]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import fastdtw
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks
from IPython.display import display, clear_output, HTML
import re
import seaborn as sns
sns.set(style="whitegrid")

import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_channel_labels.pkl')

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Remove bad files with bad labels
X = X[~(X['Label 1']=='-1') & ~(X['Label 2']=='-1')]
X.head()

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label 1,Label 2
0,CS1-2,340,"[-636, -617, -652, -560, -482, -415, -383, -46...",1,S2,af,0,0
1,CS1-2,340,"[-903.0, -873.0, -935.0, -941.0, -910.0, -845....",1,S1,af,NaN,NaN
2,CS1-2,340,"[-931.0, -896.0, -896.0, -906.0, -858.0, -839....",1,S1,af,NaN,NaN
3,CS3-4,340,"[472, 464, 491, 523, 553, 706, 1019, 1404, 164...",1,S2,af,0,0
4,CS3-4,340,"[298.0, 292.0, 303.0, 311.0, 299.0, 395.0, 451...",1,S1,af,NaN,NaN


In [3]:
# A shitty conduction delay detector
def get_delay(x, amp_thresh=None, set_thresh=False):
    if (set_thresh==True):
        if any(abs(x)>amp_thresh):
            return np.argmax(abs(x)>amp_thresh)
        else:
            return len(x)
    else:    
        return np.argmax(abs(x)>(max(abs(x))/2))
    
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp

def sample_entropy(U, m, r):

    def _maxdist(x_i, x_j):
        result = max([abs(ua-va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = np.zeros([N,m-1])
        for i in range(N-m+1):
            x[i,:] = U[i:i+m-1]

        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i != j:
                    if _maxdist(x[i,:], x[j,:]) <= r:
                        C = C + 1

        return C

    U = U/max(abs(U))
    N = len(U)

    return -np.log(_phi(m+1)/_phi(m))

def percentage_fractionation(x, peak_idxs, thresh=0.01, sr=1000):
    # Get peak indexes and amplitude
    peak_idx_diffs = np.diff(peak_idxs)
    frac_time = 0
    frac_time = np.sum(peak_idx_diffs[peak_idx_diffs < thresh*sr])
    prcnt_frac = (frac_time/len(x))*100
    return prcnt_frac

def get_local_sample_entropy(x, centre_idx, width, m=2, r=0.05):
    # Ensure width is odd
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return sample_entropy(x[:width+1], m, r)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return sample_entropy(x[len(x)-1-width:], m, r)
    else:
        return sample_entropy(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], m, r)
    
def get_location_of_max_energy(x, M=14):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    return (np.argmax(x_) + math.floor(M/2))
        
def get_local_peaks(x, centre_idx, width=25, height_thresh=0.1):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_peaks(x[:width+1], height_thresh)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_peaks(x[len(x)-1-width:], height_thresh)
    else:
        return get_peaks(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], height_thresh)
    
def get_pse(x):
    x_fft = np.fft.rfft(x)
    x_P = (1/len(x_fft))*np.absolute(x_fft)**2
    x_p = x_P/sum(x_P)
    pse = np.sum([(-p*np.log2(p)) for p in x_p])
    return pse

def get_local_pse(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_pse(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_pse(x[len(x)-1-width:])
    else:
        return get_pse(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_spectral_centroid(x):
    x_fft = np.fft.rfft(x)
    x_spectrum = np.absolute(x_fft)
    normalized_spectrum = x_spectrum/sum(x_spectrum)
    normalized_frequencies = np.arange(0, len(x_spectrum), 1)
    return sum(normalized_frequencies * normalized_spectrum)

def get_local_spectral_centroid(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_spectral_centroid(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_spectral_centroid(x[len(x)-1-width:])
    else:
        return get_spectral_centroid(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_local_energy(x, centre_idx, width=60):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return np.sum(x[:width+1]**2)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return np.sum(x[len(x)-1-width:]**2)
    else:
        return np.sum(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)]**2)
    
def get_width_max_energy(x, M=14, width_thresh=0.2):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    if any(x_[np.argmax(x_):] < width_thresh*np.max(x_)):
        end_idx = np.argmax(x_) + np.argmax(x_[np.argmax(x_):] < width_thresh*np.max(x_))
    else:
        end_idx = len(x_)-1
    if any(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_)):  
        start_idx = np.argmax(x_) - np.argmax(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_))
    else:
        start_idx = 0
    return (end_idx - start_idx)

In [4]:
def get_hand_engineered_feature_dict(x, thresh_cd=None, set_thresh_cd=False, thresh_peaks=None, set_thresh_peaks=False, show_peaks=False, col_prefix = ''):
    feature_dict = {}
    sf = max(abs(x))
    x = x/max(abs(x))

    # Hand engineered features
    if set_thresh_cd:
        thresh_cd = thresh_cd/sf
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=True'] = get_delay(x, thresh_cd, set_thresh_cd)
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    else:
        feature_dict[col_prefix + 'Conduction Delay: set_thresh=False'] = get_delay(x)
    
    height_thresh=0.1
    if set_thresh_peaks:
        thresh_peaks = thresh_peaks/sf
        peaks = get_peaks(x, height_thresh, thresh_peaks, set_thresh_peaks, plot=False)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=True'] = len(peaks[0])
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    else:
        peaks = get_peaks(x, height_thresh)
        feature_dict[col_prefix + 'Number of Peaks: set_thresh=False'] = len(peaks[0])
    
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Percentage Fractionation: thresh=0.01'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    # Denoise x for remaining features
    x = denoise(x)
    
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy: M=14'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy: width=60 r=0.025'] = get_local_sample_entropy(x, max_energy_idx, 60, m=2, r=0.025)
    feature_dict[col_prefix + 'Energy Around Max Energy'] = get_local_energy(x, max_energy_idx, 60)
    min_idx = np.argmin(x)
    max_idx = np.argmax(x)
    feature_dict[col_prefix + 'Peaks Between Min and Max'] = len([i for i in peaks[0] if ((i > min_idx) & (i < max_idx))])
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.4'] = get_width_max_energy(x, M=14, width_thresh=0.4)
    feature_dict[col_prefix + 'Width of Maximum Energy: M=14, width_thresh=0.2'] = get_width_max_energy(x, M=14, width_thresh=0.2)

    return feature_dict

def get_spectral_feature_dict(x, col_prefix = ''):
    feature_dict = {}
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))
    
    feature_dict[col_prefix + 'Power Spectral Entropy'] = get_pse(x)
    feature_dict[col_prefix + 'Spectral Centroid'] = get_spectral_centroid(x)
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=30'] = get_local_pse(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=30'] = get_local_spectral_centroid(x, max_energy_idx, width=30)
    feature_dict[col_prefix + 'Power Spectral Entropy Around Maximum Energy: width=60'] = get_local_pse(x, max_energy_idx, width=60)
    feature_dict[col_prefix + 'Spectral Centroid Around Maximum Energy: width=60'] = get_local_spectral_centroid(x, max_energy_idx, width=60)
    
    return feature_dict
    
def get_temporal_feature_dict(x, col_prefix = ''):

    feature_dict = {}
    feature_dict[col_prefix + 'Maximum Absolute Value'] = np.max(abs(x))
    
    # Denoise and normalise x for remaining features
    x = denoise(x)
    x = x/max(abs(x))


    erbc = feature_calculators.energy_ratio_by_chunks(x, [{'num_segments':10, 'segment_focus':3}, {'num_segments':10, 'segment_focus':2}])
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=2'] = erbc[1][1]
    feature_dict[col_prefix + 'Energy Ratio by Chunks: num_segments=10 segment_focus=3'] = erbc[0][1]
    feature_dict[col_prefix + 'Approximate Entropy: m=3 r=0.7'] = feature_calculators.approximate_entropy(x, 3, 0.7)
    feature_dict[col_prefix + 'Ratio Beyond 5xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 5)
    feature_dict[col_prefix + 'Ratio Beyond 4xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 4)
    feature_dict[col_prefix + 'Ratio Beyond 3xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 3)
    feature_dict[col_prefix + 'Ratio Beyond 2xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 2)
    feature_dict[col_prefix + 'Ratio Beyond 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    # A fraction q of the mass lies to the left of i. (Alternative to conduction delay?)
    imq = feature_calculators.index_mass_quantile(x, [{'q': 0.6}, {'q': 0.4}])
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.6'] = imq[0][1]
    feature_dict[col_prefix + 'Index Mass Quantile: q=0.4'] = imq[1][1]
    

    return feature_dict

In [5]:
temporal_feature_list = []
spectral_feature_list = []
hand_engineered_feature_list = []
dtw_list = []

for i,row in X[X['S1/S2']=='S2'].iterrows():
    clear_output(wait=True)
    display('Extracting Features: ' + str(round((i/X[X['S1/S2']=='S2'].index[-1])*100, 3)) + '%')
    
    # Get the patients response to the first S1 stimuli as the reference response
    ref_response = X[(X['S1/S2']=='S1') & (X['Patient']==row['Patient']) & (X['Type']==row['Type']) & (X['Channel']==row['Channel'])
                    ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    
    ref_temporal_feature_dict = get_temporal_feature_dict(ref_response['Data'], col_prefix = '')
    ref_spectral_feature_dict = get_spectral_feature_dict(ref_response['Data'], col_prefix = '')
    ref_hand_engineered_feature_dict = get_hand_engineered_feature_dict(ref_response['Data'], 
                                                                        thresh_cd=None, 
                                                                        set_thresh_cd=False, 
                                                                        thresh_peaks=max(abs(ref_response['Data'])), 
                                                                        set_thresh_peaks=True, 
                                                                        show_peaks=False, 
                                                                        col_prefix = '')
    
    temporal_feature_dict = get_temporal_feature_dict(row['Data'], col_prefix = '')
    spectral_feature_dict = get_spectral_feature_dict(row['Data'], col_prefix = '')
    hand_engineered_feature_dict = get_hand_engineered_feature_dict(row['Data'], 
                                                                    thresh_cd=None, 
                                                                    set_thresh_cd=False, 
                                                                    thresh_peaks=max(abs(ref_response['Data'])), 
                                                                    set_thresh_peaks=True, 
                                                                    show_peaks=False, 
                                                                    col_prefix = '')

    
    for (k_t, v_t), (k_s, v_s), (k_he, v_he) in zip(temporal_feature_dict.items(), spectral_feature_dict.items(), hand_engineered_feature_dict.items()):
        temporal_feature_dict[k_t] = v_t - ref_temporal_feature_dict[k_t]
        spectral_feature_dict[k_s] = v_s - ref_spectral_feature_dict[k_s]
        hand_engineered_feature_dict[k_he] = v_he - ref_hand_engineered_feature_dict[k_he]
        
    temporal_feature_dict['Label 1'] = row['Label 1']
    temporal_feature_dict['Label 2'] = row['Label 2']
    temporal_feature_dict['Channel'] = row['Channel']
    spectral_feature_dict['Label 1'] = row['Label 1']
    spectral_feature_dict['Label 2'] = row['Label 2']
    spectral_feature_dict['Channel'] = row['Channel']
    hand_engineered_feature_dict['Label 1'] = row['Label 1']
    hand_engineered_feature_dict['Label 2'] = row['Label 2']
    hand_engineered_feature_dict['Channel'] = row['Channel']
    
    fdtw = fastdtw.dtw(row['Data'], ref_response['Data'])
    dtw_dict = {}
    dtw_dict['DTW Distance'] = fdtw[0]
    dtw_dict['Label 1'] = row['Label 1']
    dtw_dict['Label 2'] = row['Label 2']
    dtw_dict['Channel'] = row['Channel']
    
    temporal_feature_list.append(temporal_feature_dict)
    spectral_feature_list.append(spectral_feature_dict)
    hand_engineered_feature_list.append(hand_engineered_feature_dict)
    dtw_list.append(dtw_dict)

'Extracting Features: 100.0%'

In [8]:
temporal_features = pd.DataFrame(temporal_feature_list)
spectral_features = pd.DataFrame(spectral_feature_list)
hand_engineered_features = pd.DataFrame(hand_engineered_feature_list)
dtw_df = pd.DataFrame(dtw_list)

In [9]:
temporal_features.head()

,Approximate Entropy: m=3 r=0.7,Channel,Energy Ratio by Chunks: num_segments=10 segment_focus=2,Energy Ratio by Chunks: num_segments=10 segment_focus=3,Index Mass Quantile: q=0.4,Index Mass Quantile: q=0.6,Label 1,Label 2,Maximum Absolute Value,Ratio Beyond 1xSTD,Ratio Beyond 2xSTD,Ratio Beyond 3xSTD,Ratio Beyond 4xSTD,Ratio Beyond 5xSTD
0,-0.020668,CS1-2,-0.025134,-0.034211,0.160000,0.193333,0,0,1889.0,0.093333,0.066667,0.046667,0.000000,0.013333
1,-0.001881,CS3-4,0.000002,-0.000017,0.106667,0.133333,0,0,7059.0,0.080000,0.066667,0.033333,0.006667,0.000000
2,-0.038883,CS5-6,-0.000478,-0.000087,0.040000,0.060000,0,0,10156.0,0.093333,0.040000,0.026667,0.000000,0.006667
3,-0.019609,CS1-2,-0.067263,-0.035227,0.153333,0.186667,0,0,1900.0,0.100000,0.066667,0.033333,0.000000,0.013333
4,-0.001023,CS3-4,-0.000239,-0.000017,0.100000,0.126667,0,0,7855.0,0.080000,0.060000,0.033333,0.006667,0.006667


In [8]:
# %matplotlib qt
# labels = ('1','0')
# colours = ['#e25238', '#6997e5']
# x_s = [-0.1, 0.1]
# features = cs12_features.drop(['Label 1', 'Label 2'], axis=1).columns

# # Make plot for each feature
# fig, axes = plt.subplots(ncols=6, nrows=math.ceil(len(features)/2), figsize=(16,9))
# for i,feature in enumerate(features):
#     if (i < math.ceil(len(features)/2)):
#         for label, colour, x in zip(labels, colours, x_s):
#             cs12_idx = (cs12_features['Label 1'].values == label)
#             cs34_idx = (cs34_features['Label 1'].values == label)
#             cs56_idx = (cs56_features['Label 1'].values == label)

#             axes[i,0].scatter(np.zeros(cs12_features.loc[cs12_idx].shape[0])+x, cs12_features.loc[cs12_idx, feature], c=colour, alpha=0.5)
#             axes[i,1].scatter(np.zeros(cs34_features.loc[cs34_idx].shape[0])+x, cs34_features.loc[cs34_idx, feature], c=colour, alpha=0.5)
#             axes[i,2].scatter(np.zeros(cs56_features.loc[cs56_idx].shape[0])+x, cs56_features.loc[cs56_idx, feature], c=colour, alpha=0.5)

#         axes[i,0].set_xlim([-0.15, 0.15])
#         axes[i,1].set_xlim([-0.15, 0.15])
#         axes[i,2].set_xlim([-0.15, 0.15])
#         axes[i,1].text(0.5, 0.9, feature, horizontalalignment='center', transform=axes[i,1].transAxes, fontsize=6)
#     else:
#         j = i-math.ceil(len(features)/2)
#         for label, colour, x in zip(labels, colours, x_s):
#             cs12_idx = (cs12_features['Label 1'].values == label)
#             cs34_idx = (cs34_features['Label 1'].values == label)
#             cs56_idx = (cs56_features['Label 1'].values == label)

#             axes[j,3].scatter(np.zeros(cs12_features.loc[cs12_idx].shape[0])+x, cs12_features.loc[cs12_idx, feature], c=colour, alpha=0.5)
#             axes[j,4].scatter(np.zeros(cs34_features.loc[cs34_idx].shape[0])+x, cs34_features.loc[cs34_idx, feature], c=colour, alpha=0.5)
#             axes[j,5].scatter(np.zeros(cs56_features.loc[cs56_idx].shape[0])+x, cs56_features.loc[cs56_idx, feature], c=colour, alpha=0.5)

#         axes[j,3].set_xlim([-0.15, 0.15])
#         axes[j,4].set_xlim([-0.15, 0.15])
#         axes[j,5].set_xlim([-0.15, 0.15])
#         axes[j,4].text(0.5, 0.9, feature, horizontalalignment='center', transform=axes[j,4].transAxes, fontsize=6)
        
# axes[0,0].set_title('CS1-2', fontsize=8)  
# axes[0,1].set_title('CS3-4', fontsize=8)  
# axes[0,2].set_title('CS5-6', fontsize=8)  
# axes[0,3].set_title('CS1-2', fontsize=8)  
# axes[0,4].set_title('CS3-4', fontsize=8)  
# axes[0,5].set_title('CS5-6', fontsize=8)  
# plt.subplots_adjust(hspace = 0.2, bottom=0.05, top=0.95, left=0.05, right=0.95)
# plt.show()

In [9]:
%matplotlib qt
feature_names = temporal_features.drop(['Label 1', 'Label 2', 'Channel'], axis=1).columns

sns.set(style="whitegrid")
# Make plot for each feature
fig, axes = plt.subplots(ncols=2, nrows=math.ceil(len(feature_names)/2), figsize=(16,9))
plt.rcParams.update({'font.size': 6})
for i,feature in enumerate(feature_names):
    if (i < math.ceil(len(feature_names)/2)):
        axes[i,0] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=temporal_features, ax=axes[i,0])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[i,0].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[i,0].set(xlabel='')
    else:
        j = i - math.ceil(len(feature_names)/2)
        axes[j,1] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=temporal_features, ax=axes[j,1])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[j,1].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[j,1].set(xlabel='')
        
plt.show()

In [37]:
%matplotlib qt
feature_names = spectral_features.drop(['Label 1', 'Label 2', 'Channel'], axis=1).columns

sns.set(style="whitegrid")
# Make plot for each feature
fig, axes = plt.subplots(ncols=2, nrows=math.ceil(len(feature_names)/2), figsize=(16,9))
plt.rcParams.update({'font.size': 6})
for i,feature in enumerate(feature_names):
    if (i < math.ceil(len(feature_names)/2)):
        axes[i,0] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=spectral_features, ax=axes[i,0])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[i,0].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[i,0].set(xlabel='')
    else:
        j = i - math.ceil(len(feature_names)/2)
        axes[j,1] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=spectral_features, ax=axes[j,1])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[j,1].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[j,1].set(xlabel='')
        
plt.show()

In [9]:
%matplotlib qt
feature_names = hand_engineered_features.drop(['Label 1', 'Label 2', 'Channel'], axis=1).columns

sns.set(style="whitegrid")
# Make plot for each feature
fig, axes = plt.subplots(ncols=2, nrows=math.ceil(len(feature_names)/2), figsize=(16,9))
plt.rcParams.update({'font.size': 6})
for i,feature in enumerate(feature_names):
    if (i < math.ceil(len(feature_names)/2)):
        axes[i,0] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=hand_engineered_features, ax=axes[i,0])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[i,0].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[i,0].set(xlabel='')
    else:
        j = i - math.ceil(len(feature_names)/2)
        axes[j,1] = sns.violinplot(x='Channel', y=feature, hue='Label 1', split=True, palette='Set2', data=hand_engineered_features, ax=axes[j,1])
        feature_ylabel = re.sub(r'(:)', r'\1\n', feature)
        axes[j,1].set_ylabel(ylabel=feature_ylabel, fontsize=8)
        axes[j,1].set(xlabel='')
        
plt.show()

In [12]:
%matplotlib qt

sns.set(style="whitegrid")
# Make plot for each feature
fig = plt.figure(figsize=(16,9))
ax = sns.swarmplot(x='Channel', y='Number of Peaks: set_thresh=False', hue='Label 1', split=True, palette='Set2', data=hand_engineered_features)
plt.title('Detected Number of Peaks')
plt.show()

/anaconda3/lib/python3.6/site-packages/seaborn/categorical.py:2783: UserWarning: The `split` parameter has been renamed to `dodge`.
  warnings.warn(msg, UserWarning)
